In [27]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Dropout, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Dropout, MaxPooling2D
from keras import regularizers


In [54]:
_rotate_range = 180
_size = (128, 128)
_batch_size = 32
_filters = (4, 4)
_epochs = 30
_regularizers = 0.0001
_probability_to_change = 0.30
_num_class=17

def create_model():
        model = Sequential()

        model.add(Conv2D(64, _filters, input_shape=(_size[0], _size[1], 3), padding='same',
                        kernel_regularizer=regularizers.l1_l2(_regularizers, _regularizers), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(64, _filters, kernel_regularizer=regularizers.l2(_regularizers), 
                padding='same',activation='relu'))
        model.add(MaxPooling2D())
        model.add(Dropout(0.20))

        model.add(Conv2D(128, _filters, kernel_regularizer=regularizers.l2(_regularizers), 
                padding='same',activation='relu'))
        model.add(Conv2D(128, _filters, kernel_regularizer=regularizers.l2(_regularizers),
                padding='same',activation='relu'))
        model.add(MaxPooling2D())
        model.add(Dropout(0.20))

        model.add(Conv2D(256, _filters, kernel_regularizer=regularizers.l2(_regularizers), 
                padding='same',activation='relu'))
        model.add(Conv2D(256, _filters, kernel_regularizer=regularizers.l2(_regularizers), 
                padding='same',activation='relu'))
        model.add(MaxPooling2D())
        model.add(Dropout(0.20))

        model.add(Flatten())
        model.add(Dense(512, kernel_regularizer=regularizers.l2(_regularizers), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(_num_class, activation='softmax'))

        model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
        
        return model
model=create_model()
print(model.summary())


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_44 (Conv2D)          (None, 128, 128, 64)      3136      
                                                                 
 batch_normalization_7 (Batc  (None, 128, 128, 64)     256       
 hNormalization)                                                 
                                                                 
 conv2d_45 (Conv2D)          (None, 128, 128, 64)      65600     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 64, 64, 64)       0         
 g2D)                                                            
                                                                 
 dropout_28 (Dropout)        (None, 64, 64, 64)        0         
                                                                 
 conv2d_46 (Conv2D)          (None, 64, 64, 128)      

In [58]:
from keras.utils import image_dataset_from_directory
datadir = './Dataset'
train_ds = image_dataset_from_directory(
    datadir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=_size,
    batch_size=_batch_size
)
val_ds = image_dataset_from_directory(
    datadir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=_size,
    batch_size=_batch_size
)
class_names = train_ds.class_names
print(len(class_names))


Found 10469 files belonging to 17 classes.
Using 8376 files for training.
Found 10469 files belonging to 17 classes.
Using 2093 files for validation.
17


In [56]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [46]:
a = list(train_ds.as_numpy_iterator())[0]

In [52]:
a[0].shape

(32, 128, 128, 3)

In [57]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=_epochs
)


Epoch 1/30


2023-02-24 12:49:02.792956: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_8/dropout_28/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-24 12:49:13.774364: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-24 12:49:19.331232: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 274.60MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-02-24 12:49:19.476287: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 274.60MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2

KeyboardInterrupt: 